In [13]:
{
    "tags": [
        "hide-input",
    ]
}
!{sys.executable} -m pip install  pygsheets
!{sys.executable} -m pip install --upgrade pyarrow
#get data
import pygsheets
import configparser
import sys
import snowflake.connector
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import numpy as np                   # v 1.19.2
import matplotlib.pyplot as plt      # v 3.3.2
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
# calculate the net_arr bucket of open deals
import seaborn as sns
from math import floor 
#NF Jupyter is not recognizing the right folder. Needed to set it up manually
import os

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from gitlabdata.orchestration_utils import (
    data_science_engine_factory,
    query_dataframe,
)

engine = data_science_engine_factory()

service_file_path = '/Volumes/GoogleDrive-101341343143168722397/My Drive/docker_work/jupyter_analysis/20220210_FY22_channel_value_analysis/nfiguera-c3fe9e64-a6543dd51e79.json'

#target gsheet for uploads and downloads, just allowing one to keep it more secure
#spreadsheet_id = '1-TO3GATTq240IOafNqZTAi9ixmvdeDUzotAYlSMtFeU'
spreadsheet_id = '1Vwu8euxRgIF3QYWK8hAbp4Vy21AlFfpDwI4MaEEiIWk'



# Get the current working directory
cwd = os.getcwd()
print(cwd)
os.chdir("/Volumes/GoogleDrive-101341343143168722397/My Drive/docker_work/jupyter_analysis/20220223_Targets_&_CoverageCurves/")

#how to add access to gsheet
#https://stackoverflow.com/questions/62917910/python-export-pandas-dataframe-to-google-sheets-solved

/Volumes/GoogleDrive-101341343143168722397/My Drive/docker_work/jupyter_analysis/20220223_Targets_&_CoverageCurves


In [8]:
import os as os
os.getcwd()

'/Volumes/GoogleDrive-101341343143168722397/My Drive/docker_work/jupyter_analysis/20220223_Targets_&_CoverageCurves'

# Helper Functions

In [9]:
#######
## Create a fitter curve of the last 4 quarters for comparison
#######
def objective(x, a, b, c, d, e):
    return a * x + b * x**2 + c * x**3 + d * x**4 + e


def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1

def read_from_gsheet(service_file_path, spreadsheet_id, sheet_name):
    """
    this function takes a sheet_name from a spreadsheet_id and returns a data frame 
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
   
    wks_read = sh.worksheet_by_title(sheet_name)
    read = wks_read.get_as_df()
    
    return read


def run_query_in_snowflake(conn, sql):
    cur = conn.cursor()
    cur.execute(sql)
    df = cur.fetch_pandas_all()
    return df

def executeScriptFromFile(filename, engine):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()
    
    results = -1
    
    try:
        results = query_dataframe(engine,sqlFile)
    except:
        print("Command did not run")
    
    return results

###########################################################

def days_between(d1, d2):
    #d1 = datetime.strptime(d1, "%Y-%m-%d")
    #d2 = datetime.strptime(d2, "%Y-%m-%d")
    return (d2 - d1).days


def calculate_quarters_after_creation(x):
        
    age = 0
    
    if (x['IS_OPEN'] == 1):
        age = days_between(x['CREATED_FISCAL_QUARTER_DATE'], x['SNAPSHOT_FISCAL_QUARTER_DATE'])
    elif (x['IS_OPEN']== 0 and x['SNAPSHOT_DATE'] <= x['CLOSE_DATE']):
        age = days_between(x['CREATED_FISCAL_QUARTER_DATE'], x['SNAPSHOT_FISCAL_QUARTER_DATE'])
    else:
        age = days_between(x['CREATED_FISCAL_QUARTER_DATE'], x['CLOSE_FISCAL_QUARTER_DATE'])
    
    quarter_delta = floor(age/90)
    
    return quarter_delta

def calculate_channel_track (x):
    
    channel_track = 'Direct'
    
    if (x['deal_path'] == 'Direct'):
        channel_track = 'Direct'
    elif (x['deal_path'] == 'Web Direct'):
        channel_track = 'Web Direct'
    elif (x['deal_path'] == 'Channel'
        and x['sales_qualified_source'] != 'Channel Generated'): 
        channel_track = 'Partner Co-Sell'
    elif (x['deal_path'] == 'Channel'):
        channel_track = 'Partner Sourced'
    
    return channel_track

In [10]:
# calculates pending coverage using a minimum of 50k pending
def calculate_to_pending_coverage (x, metric, qtd, actual, minimum_delta=5000):
    
    result = None
    
    actual = float(x[actual])
    qtd = float(x[qtd])
    metric = float(x[metric])
    
    if (actual - qtd) > minimum_delta:
        result = metric / (actual - qtd)
        result = min(result,6) # limiting the maximum amount of coverage to account for noise in the models
    
    return result

# calculates pending coverage using a minimum of 50k pending
def calculate_bookings_linearity (x, qtd_bookings, actual_booked):
    
    result = None
    
    actual = float(x[actual_booked])
    qtd = float(x[qtd_bookings])
       
    if actual > 0:
        result = qtd / actual 
        
    return result 


# fits a curve to the subset data using the defined objective function
def fit_curve_to_agg (data_agg, x_label, y_label):
    
    # fit a curve
    x, y = data_agg[x_label], data_agg[y_label]
    # curve fit
    popt, _ = curve_fit(objective, x, y)

    x_line = np.arange(min(x), max(x), 1)
    # calculate the output for the range
    # summarize the parameter values
    a, b, c , d, e = popt
    y_line = objective(x_line, a, b, c, d, e)
  
    curve_result = pd.DataFrame({x_label:x_line,y_label:y_line})
    return curve_result

In [11]:
def opportunity_source_to_sqs(df):
    
    result = None
    
    if df['opportunity_source'] is not None:
    
        result = df['opportunity_source'].lower()

        if df['opportunity_source'].lower() == 'partner generated':
            result = 'channel generated'
        elif df['opportunity_source'].lower() =='n/a':
            result = None
        elif df['opportunity_source'].lower() == 'web generated':
            result = 'web direct generated'
    
    return result

def order_type_to_deal_group (df):
    
    result = None
    
    if df['order_type'] is not None:
        
        if df['order_type'] == '1. New - First Order':
            result = '1. new'
        elif df['order_type'] == '3. Growth':
            result = '2. growth'
    
    return result

## Data Collection

In [21]:
sheet_name = 'topo_targets_ssot'
ssot_targets_topo = read_from_gsheet(service_file_path, spreadsheet_id, sheet_name)

In [27]:
ssot_targets_topo.columns

Index([                       'kpi_name',                           'month',
                    'opportunity_source',                      'order_type',
               'segment_geo_region_area',                'allocated_target',
                               'segment',                             'geo',
                                'region',                            'area',
                         'close_quarter',                       'month_num',
                  'Logos Segment Target',               'Opp_source_lookup',
                   'allocated_target_ur',                             0.411,
                     'Monthly Logic End',               'Monthly Logic New',
                  'Monthly Logic Expand',                        'Cocenate',
                           'key_overall',                         'key_sqs',
                                'key_ot',                     'key_segment',
                        'key_segment_ot',                 'key_segment_sqs',

In [23]:
# standardize sales qualified source
ssot_targets_topo['opportunity_source'] = ssot_targets_topo.apply(opportunity_source_to_sqs,axis=1)
ssot_targets_topo.opportunity_source.unique()

#rename quarter fields
ssot_targets_topo.rename({'Quarter':'close_quarter'},axis=1,inplace=True)

################################

# TOPO keys are not exactly as they should be in SFDC 
# this part is accounting for that, it might change in the future

# fix pubsec geo and region
index = ssot_targets_topo['segment'].astype(str).str.lower() == 'pubsec'
ssot_targets_topo.loc[index,'geo'] = 'amer'

# change mid-market region to AMER on AMER geo
index = (ssot_targets_topo['segment'].astype(str).str.lower() == 'mid-market') & (ssot_targets_topo['geo'].astype(str).str.lower() == 'amer')
ssot_targets_topo.loc[index,'region'] = 'amer'

# change mid-market region to EMEA on EMEA geo
index = (ssot_targets_topo['segment'].astype(str).str.lower() == 'mid-market') & (ssot_targets_topo['geo'].astype(str).str.lower() == 'emea')
ssot_targets_topo.loc[index,'region'] = 'emea'

# change SMB region to AMER on AMER geo
index = (ssot_targets_topo['segment'].astype(str).str.lower() == 'smb') & (ssot_targets_topo['geo'].astype(str).str.lower() == 'amer')
ssot_targets_topo.loc[index,'region'] = 'amer'

# change SMB region to EMEA on EMEA geo
index = (ssot_targets_topo['segment'].astype(str).str.lower() == 'smb') & (ssot_targets_topo['geo'].astype(str).str.lower() == 'emea')
ssot_targets_topo.loc[index,'region'] = 'emea'

################################

# change smb region to amer

# standardize order type
ssot_targets_topo['order_type'] = ssot_targets_topo.apply(order_type_to_deal_group,axis=1)
ssot_targets_topo.order_type.unique()

# standardize segment, geo, region, area
ssot_targets_topo['segment'] = ssot_targets_topo['segment'].map(lambda x: x.lower())
ssot_targets_topo['geo'] = ssot_targets_topo['geo'].astype(str).str.lower()
ssot_targets_topo['region'] = ssot_targets_topo['region'].astype(str).str.lower()
ssot_targets_topo['area'] = ssot_targets_topo['area'].astype(str).str.lower()    

# create standard keys
ssot_targets_topo['key_overall'] = 'global'

ssot_targets_topo['key_sqs'] = ssot_targets_topo['opportunity_source']
ssot_targets_topo['key_ot'] = ssot_targets_topo['order_type'] 

ssot_targets_topo['key_segment'] = ssot_targets_topo['segment']
ssot_targets_topo['key_segment_ot'] = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['order_type'] 
ssot_targets_topo['key_segment_sqs'] = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['opportunity_source'] 

ssot_targets_topo['key_segment_geo'] = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['geo']
ssot_targets_topo['key_segment_geo_ot'] = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['geo'] + '_' + ssot_targets_topo['order_type'] 
ssot_targets_topo['key_segment_geo_sqs'] = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['geo'] + '_' + ssot_targets_topo['opportunity_source'] 

ssot_targets_topo['key_segment_geo_region']     = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['geo'] + '_' + ssot_targets_topo['region'] 
ssot_targets_topo['key_segment_geo_region_ot']  = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['geo'] + '_' + ssot_targets_topo['region'] + '_' + ssot_targets_topo['order_type'] 
ssot_targets_topo['key_segment_geo_region_sqs'] = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['geo'] + '_' + ssot_targets_topo['region'] + '_' + ssot_targets_topo['opportunity_source'] 

ssot_targets_topo['key_segment_geo_region_area']     = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['geo'] + '_' + ssot_targets_topo['region'] + '_' + ssot_targets_topo['area']
ssot_targets_topo['key_segment_geo_region_area_ot']  = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['geo'] + '_' + ssot_targets_topo['region'] + '_' + ssot_targets_topo['area'] + '_' + ssot_targets_topo['order_type'] 
ssot_targets_topo['key_segment_geo_region_area_sqs'] = ssot_targets_topo['segment'] + '_' + ssot_targets_topo['geo'] + '_' + ssot_targets_topo['region'] + '_' + ssot_targets_topo['area'] + '_' + ssot_targets_topo['opportunity_source']

In [26]:
ssot_targets_topo

,kpi_name,month,opportunity_source,order_type,segment_geo_region_area,allocated_target,segment,geo,region,area,...,key_segment_sqs,key_segment_geo,key_segment_geo_ot,key_segment_geo_sqs,key_segment_geo_region,key_segment_geo_region_ot,key_segment_geo_region_sqs,key_segment_geo_region_area,key_segment_geo_region_area_ot,key_segment_geo_region_area_sqs
0,Deals,FY23-Feb,sdr generated,1. new,Large-AMER-East-MATL,0.384019,large,amer,east,matl,...,large_sdr generated,large_amer,large_amer_1. new,large_amer_sdr generated,large_amer_east,large_amer_east_1. new,large_amer_east_sdr generated,large_amer_east_matl,large_amer_east_matl_1. new,large_amer_east_matl_sdr generated
1,Deals,FY23-Feb,sdr generated,1. new,Large-AMER-East-NE,0.269858,large,amer,east,ne,...,large_sdr generated,large_amer,large_amer_1. new,large_amer_sdr generated,large_amer_east,large_amer_east_1. new,large_amer_east_sdr generated,large_amer_east_ne,large_amer_east_ne_1. new,large_amer_east_ne_sdr generated
2,Deals,FY23-Feb,sdr generated,1. new,Large-AMER-East-NY,0.282658,large,amer,east,ny,...,large_sdr generated,large_amer,large_amer_1. new,large_amer_sdr generated,large_amer_east,large_amer_east_1. new,large_amer_east_sdr generated,large_amer_east_ny,large_amer_east_ny_1. new,large_amer_east_ny_sdr generated
3,Deals,FY23-Feb,sdr generated,1. new,Large-AMER-East-SCEN,0.213953,large,amer,east,scen,...,large_sdr generated,large_amer,large_amer_1. new,large_amer_sdr generated,large_amer_east,large_amer_east_1. new,large_amer_east_sdr generated,large_amer_east_scen,large_amer_east_scen_1. new,large_amer_east_scen_sdr generated
4,Deals,FY23-Feb,sdr generated,1. new,Large-AMER-East-SE,0.740607,large,amer,east,se,...,large_sdr generated,large_amer,large_amer_1. new,large_amer_sdr generated,large_amer_east,large_amer_east_1. new,large_amer_east_sdr generated,large_amer_east_se,large_amer_east_se_1. new,large_amer_east_se_sdr generated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10994,Expansion Amount,FY23-Apr,,2. growth,Large-APAC-APAC-APAC,,large,apac,apac,apac,...,large_,large_apac,large_apac_2. growth,large_apac_,large_apac_apac,large_apac_apac_2. growth,large_apac_apac_,large_apac_apac_apac,large_apac_apac_apac_2. growth,large_apac_apac_apac_
10995,Expansion Amount,FY23-May,,2. growth,Large-APAC-APAC-APAC,,large,apac,apac,apac,...,large_,large_apac,large_apac_2. growth,large_apac_,large_apac_apac,large_apac_apac_2. growth,large_apac_apac_,large_apac_apac_apac,large_apac_apac_apac_2. growth,large_apac_apac_apac_
10996,Expansion Amount,FY23-Jun,,2. growth,Large-APAC-APAC-APAC,,large,apac,apac,apac,...,large_,large_apac,large_apac_2. growth,large_apac_,large_apac_apac,large_apac_apac_2. growth,large_apac_apac_,large_apac_apac_apac,large_apac_apac_apac_2. growth,large_apac_apac_apac_
10997,Expansion Amount,FY23-Jul,,2. growth,Large-APAC-APAC-APAC,,large,apac,apac,apac,...,large_,large_apac,large_apac_2. growth,large_apac_,large_apac_apac,large_apac_apac_2. growth,large_apac_apac_,large_apac_apac_apac,large_apac_apac_apac_2. growth,large_apac_apac_apac_


In [28]:
ssot_targets_topo.key_segment_geo_region_area.unique()

array(['large_amer_east_matl', 'large_amer_east_ne', 'large_amer_east_ny',
       'large_amer_east_scen', 'large_amer_east_se', 'large_amer_west_sw',
       'large_amer_west_ncen', 'large_amer_west_nw',
       'large_emea_cemea_cemea', 'large_emea_dach_dach',
       'large_emea_neur_norben', 'large_emea_neur_uk/i',
       'large_emea_seur_seur', 'large_apac_apac_anz',
       'large_apac_apac_japan', 'large_apac_apac_korea',
       'large_apac_apac_seat', 'pubsec_amer_pubsec_fsi',
       'pubsec_amer_pubsec_dod', 'pubsec_amer_pubsec_civilian',
       'pubsec_amer_pubsec_nsg', 'pubsec_amer_pubsec_sled',
       'mid-market_amer_amer_central', 'mid-market_amer_amer_northeast',
       'mid-market_amer_amer_southeast', 'mid-market_amer_amer_west',
       'mid-market_emea_emea_central', 'mid-market_emea_emea_north',
       'mid-market_emea_emea_south', 'mid-market_apac_apac_apac',
       'smb_amer_amer_east', 'smb_amer_amer_west',
       'smb_emea_emea_central', 'smb_emea_emea_north',
       

In [29]:
# test the overall number works
index = ssot_targets_topo.kpi_name == 'Net ARR'
ssot_targets_topo[index].groupby(['key_segment_ot']).allocated_target.sum()

key_segment_ot
large_1. new            12015784.548783
large_2. growth         93383739.257451
mid-market_1. new       10069064.868061
mid-market_2. growth    41931317.750224
pubsec_1. new            1351843.833613
pubsec_2. growth         24447917.64029
smb_1. new               8721703.192456
smb_2. growth           18078628.250954
Name: allocated_target, dtype: object

In [30]:
# create the preaggregated targets per level
agg_key_list = ['key_segment','key_overall','key_sqs', 'key_ot','key_segment_sqs', 'key_segment_ot', 'key_segment_geo', 'key_segment_geo_sqs',
       'key_segment_geo_ot', 'key_segment_geo_region',
       'key_segment_geo_region_sqs', 'key_segment_geo_region_ot',
       'key_segment_geo_region_area', 'key_segment_geo_region_area_sqs',
       'key_segment_geo_region_area_ot']


results = None
for agg_key in agg_key_list:
    
    temp = ssot_targets_topo.groupby(['kpi_name','close_quarter']+[agg_key])['allocated_target'].sum().reset_index()
    
    temp.rename({agg_key:'agg_key_value'}, axis=1, inplace=True)
    temp['agg_key_name'] = agg_key
    
    if results is None:
        results = temp.copy()
    else:
        results = results.append(temp)

results.columns


Index(['kpi_name', 'close_quarter', 'agg_key_value', 'allocated_target',
       'agg_key_name'],
      dtype='object')

In [31]:
#### PIVOT QUARTERS so the table is already in the right shape
to_gsheet = results.pivot(index=['kpi_name', 'agg_key_name', 'agg_key_value'], columns='close_quarter', values='allocated_target').reset_index()
to_gsheet.columns

Index(['kpi_name', 'agg_key_name', 'agg_key_value', 'FY23-Q1', 'FY23-Q2',
       'FY23-Q3', 'FY23-Q4'],
      dtype='object', name='close_quarter')

### Save to Gsheets

Links to the file:

https://docs.google.com/spreadsheets/d/1Vwu8euxRgIF3QYWK8hAbp4Vy21AlFfpDwI4MaEEiIWk/edit#gid=930691697

In [32]:
#capture regions to country
#only execute if we want to run the gsheet upload / download
sheet_name = 'ssot_targets_pre_aggregated'

index = to_gsheet.kpi_name == 'Net ARR'

fields_order = ['kpi_name', 'agg_key_name', 'agg_key_value', 'FY23-Q1', 'FY23-Q2','FY23-Q3', 'FY23-Q4']
write_to_gsheet(service_file_path, spreadsheet_id, sheet_name,to_gsheet[index][fields_order])